In [1]:
prompt_system = """You are a task judge. You need to judge whether this task can be executed locally.
This task should be like a task, not subjective considerations. For example, if there are 'custom', 'you want' and other situations, they cannot be considered and should return false and be classified as Non_task. Any subjective will carsh the system.
This task should specific the element, for example, if there only are 'text' without the specific string, they cannot be considered and should return false and be classified as Non_task.
This task should not involve interactions with other application plug-ins, etc., and only rely on Word. If 'Excel', 'Edge' and other interactions are involved, it should return false and be classified as App_involve.
This task should not involve version updates and other interactions that depend on the environment, but only rely on the current version, and do not want to be upgraded or downgraded. It should return false and be classified as Env.
There are other things that you think cannot be executed or are irrelevant, return False, and be classified as Others

Your response should be strictly structured in a JSON format, consisting of three distinct parts with the following keys and corresponding content:
    {
      "Judge": <true or false depends on you think this task whether can be performed.>
      "Thought": <Outline the reason why you give the judgement.>
      "Type":<None/Non_task/App_involve/Env/Others>
    }
Make sure you answer must be strictly in JSON format only, without other redundant text such as json header. Otherwise it will crash the system.

Below is only a example of the response. Do not fall in the example.


"""
prompt_example = r"""
example1:
    -- User_input: {"Change the type of the text"}

    -- Response: {
        "Judge": false,
        "Thought": "Not specific the type of the text you want to change to, and not specific the text"
        "Type": "Non_task"
        }
example2:
    -- User_input: {"Cut and paste the text in Word"}

    -- Response: {
        "Judge": false,
        "Thought": "Not specific the text."
        "Type": "Non_task"
        }
example3:
    -- User_input: {"Cut and paste the text 'text to edit' in Word"}

    -- Response: {
        "Judge": true,
        "Thought": "Good to take."
        "Type": "None"
        }

"""

In [2]:
from azure.identity import ManagedIdentityCredential
from openai import AzureOpenAI
identity = ManagedIdentityCredential(
    client_id="14e8afd2-92b1-4b48-9c89-97a095be4895"
)
api_scope = "api://" + "feb7b661-cac7-44a8-8dc1-163b63c23df2" + "/.default"
def get_token():
    return identity.get_token(api_scope).token

client: AzureOpenAI = AzureOpenAI(
    max_retries=10,
    timeout=10,
    api_version="2024-04-01-preview",
    azure_endpoint="https://cloudgpt-openai.azure-api.net/",
    azure_ad_token_provider=get_token,
)


def chat(ques, model = "gpt-4o-20240513"):
    if not ques:
        return
    message = [{"role": "system","content":[{"type":"text","text": prompt_system + prompt_example}]},
    {
        "role": "user",
        # multi-modality input in message
        "content": [
            {
                "type": "text",
                "text": ques,
            },
            
            
        ]
    }
]
    count = 0
    if count < 5:
        count +=1
        try:
            response = client.chat.completions.create(
                model=model,
                messages=message,
            )
            return response.choices[0].message.content
        except:
            pass

In [3]:
path = r"D:\agents\UFO\sample\sample_7\sample\tasks".replace('\\','\\\\')
import os, json
dicts = {}
values = {}
for file in os.listdir(path):
    name,_ = os.path.splitext(file)
    dicts[name] = False
    values[name] = json.load(open(os.path.join(path, file)))
json.dump(dicts, open('all.json', 'w'))

In [4]:
_dict = {}
def get_answer(name):
    ques = values.get(name)
    que = ques.get('new_problem')
    answer = chat(que)
    _dict[name] = answer if answer else 'No response'
import concurrent.futures
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(get_answer, name) for name in dicts]
    results = [future.result() for future in concurrent.futures.as_completed(futures)]

json.dump(_dict, open('new_all.json', 'w'))


FileNotFoundError: [Errno 2] No such file or directory: 'new_all,json'

In [5]:
json.dump(_dict, open('new_all.json', 'w'))

In [1]:
import json
_dict = json.load(open('new_all.json', 'r'))
non = 0
app = 0
env = 0
oth = 0
ffalse = 0
for key,value in _dict.items():
    try:
        value = json.loads(value)
    except:
        print(value)
        ffalse += 1
        continue
    if value.get('Type') == 'Non_task':
        non += 1
    elif value.get('Type') == 'App_involve':
        app += 1
    elif value.get('Type') == 'Env':
        env += 1
    elif value.get('Type') == 'Others':
        oth += 1
print(f"Non_task: {non}, App_involve: {app}, Env: {env}, Others: {oth}, ffal: {ffalse}")

No response
No response
No response
{
    "Judge": false,
    "Thought": "Not specific the content of the report."
    "Type": "Non_task"
}
No response
No response
{
    "Judge": true,
    "Thought": "The task is specific and can be executed locally using Word features to change the document layout to a single page."
    "Type": "None"
}
{
    "Judge": true,
    "Thought": "The task of creating an abbreviation list by inserting a Table of Authorities can be performed locally within Word without involving other applications or relying on subjective considerations."
    "Type": "None"
}
No response
No response
{
    "Judge": false,
    "Thought": "Not specific the color or the specific header and footer."
    "Type": "Non_task"
}
{
    "Judge": true,
    "Thought": "The task specifies creating a non-conformance report template with a title and a labeled table, and then saving it as a template, all of which can be done in Word."
    "Type": "None"
}
{
    "Judge": true,
    "Thought": "Sp

In [26]:
# import shutil
length = len(_dict)
count = 0
new_path = os.path.join(path, os.pardir, 'task')
os.makedirs(new_path, exist_ok=True)
for key,value in _dict.items():
    try:
        if json.loads(value).get('Judge'):
            tmp = {}
            count +=1
            # shutil.copy(os.path.join(path, key+'.json'), os.path.join(new_path, key+'.json'))
            tmp = json.load(open(os.path.join(path, key+'.json'), 'r'))
            tmp['action_prefill_file_path'] = os.path.join(new_path, os.pardir, 'files', key+'.docx')
            json.dump(tmp, open(os.path.join(new_path, key+'.json'),'w'))
    except:
        # print(os.path.join(path, key+".json"))
        pass
    # assert 0
print(length)
print(count)

2670
1288


D:\\agents\\UFO\\sample\\sample_7\\sample\\tasks\..\task
D:\\agents\\UFO\\sample\\sample_7\\sample\\tasks\..
